In [7]:
# 处理原始数据
import pandas as pd
import os
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 281)
pd.set_option('display.width', None)

# 场景
benbu = 'benbu_baseline'
shangjin = 'shangjin_baseline'
tianfu = 'tianfu_baseline'
wuhou = 'wuhou_baseline'

In [8]:
def process_xlsx_data(file_name: str) -> None:
    """ 处理xlsx数据，去掉id列，去掉缺失值行，one-hot编码
    
    Args:
        file_name (str): 文件名，不带后缀
    """
    data = pd.read_excel(f'data_original/{file_name}.xlsx', keep_default_na=False, na_values=[""])
    print(data.shape)
    # 去掉id列，将列排序
    data = data[sorted(data.columns)]
    print(data.columns)
    del_col = ['id_patientarchive', "id_patient"]
    cols_to_drop = [c for c in del_col if c in data.columns]
    data.drop(columns=cols_to_drop, inplace=True)
    # 去除有缺失值的行
    print(data.columns)
    missing = data.isna().sum()
    print(missing)
    data.dropna(inplace=True)
    missing = data.isna().sum()
    print(missing)
    print(f'还剩{data.shape[0]}样本')
    os.makedirs('data_processed', exist_ok=True)
    output_path = f'data_processed/{file_name}_cleaned.csv'
    particular_col = ['Alcohol', 'BPH', 'Diabete', 'Hypertension', 'Smoke', 'Stone_history', 'marriage_simplified', 'nation', 'occupation']
    data.to_csv(output_path, index=False, encoding='utf-8-sig')
    df = pd.get_dummies(data, columns=particular_col, dtype='int8')
    output_path = f'data_processed/{file_name}_cleaned_onehot.csv'
    df.to_csv(output_path, index=False, encoding='utf-8-sig')
    print(df.shape)

In [ ]:
process_xlsx_data(benbu)
process_xlsx_data(shangjin)
process_xlsx_data(tianfu)
process_xlsx_data(wuhou)

(10000, 81)
Index(['ABSI', 'AKP', 'ALB_to_GLO_ratio', 'ALT', 'AST', 'AST_to_ALT_ratio',
       'Age', 'Albumin', 'Alcohol', 'BLR', 'BMI', 'BPH', 'BRI',
       'Basophil_count', 'Basophil_percentage', 'CK', 'Cholesterol',
       'Creatinine', 'Cystatin.C', 'DBIL', 'Diabete', 'Diastolic_BP', 'ELR',
       'Eosinophil_count', 'Eosinophil_percentage', 'Fasting_blood_glucose',
       'GGT', 'Gender', 'Globulin', 'HB', 'HCT', 'HDL', 'Height', 'Hips',
       'Hypertension', 'IBIL', 'LDH', 'LDL', 'Lymphocytes_count',
       'Lymphocytes_percentage', 'MCH', 'MCHC', 'MCV', 'MLR',
       'Monocytes_count', 'Monocytes_percentage', 'NLR', 'Neutrophil_count',
       'Neutrophil_percentage', 'PLR', 'RBC', 'RDW.CV', 'RDW.SD', 'Serum_Urea',
       'Serum_Uric_acid', 'Smoke', 'Stone_history', 'Systolic_BP', 'TBIL',
       'Total_protein', 'Triglycerides', 'Uric_PH', 'Uric_RBC', 'Uric_WBC',
       'Uric_bacteria', 'Uric_conductivity', 'Uric_epithelium',
       'Uric_specific_gravity', 'WBC', 'Waist_to_hi

: 

In [3]:
import pandas as pd
import numpy as np
from typing import List, Tuple

def generate_medical_samples(n_samples: int = 10000, random_state: int = 42) -> pd.DataFrame:
    """
    生成医疗数据样本
    
    参数:
        n_samples: 要生成的样本数量
        random_state: 随机种子
    
    返回:
        包含生成数据的DataFrame
    """
    np.random.seed(random_state)
    
    # 定义分类变量的可能值
    categorical_mappings = {
        'Smoke': ['Smoker', 'Never', 'Unknown',],
        'Alcohol': ['Drinker', 'Never', 'Unknown', ],
        'Gender': [0, 1],
        'stone': [True, False],
        'Hypertension': ['None Hypertension', 'Hypertension', 'Unknown'],
        'Diabete': [ 'Diabete', 'None', 'Unknown'],
        'Stone_history': ['None', 'Former Stone Carrier',  'Unknown'],
        'BPH': ['None', 'BPH', 'Unknown'],
        'nation': ['Han People', 'Minority Ethnic Groups', 'Unknown'],
        'occupation': ['No Manual Laborer', 'Manual Laborer', 'Unknown'],
        'marriage_simplified': ['married', 'unmarried', 'Unknown']
    }
    
    data = {}
    
    # 生成ID
    data['id_patientarchive'] = [f'ARCH{i+1:06d}' for i in range(n_samples)]
    data['id_patient'] = [f'P{i+1001:04d}' for i in range(n_samples)]
    
    # 生成连续型变量
    # 肾功能指标
    data['Cystatin.C'] = np.random.uniform(0.6, 1.5, n_samples).round(2)
    data['Creatinine'] = np.random.uniform(45, 120, n_samples).round(0)
    data['Serum_Uric_acid'] = np.random.uniform(150, 500, n_samples).round(0)
    data['Serum_Urea'] = np.random.uniform(2.5, 8.5, n_samples).round(2)
    
    # 尿液检查指标
    data['Uric_RBC'] = np.random.choice([0, 1, 2, 3, 4, 5], n_samples, p=[0.5, 0.25, 0.15, 0.06, 0.03, 0.01])
    data['Uric_WBC'] = np.random.choice([0, 1, 2, 3, 4], n_samples, p=[0.6, 0.25, 0.1, 0.04, 0.01])
    data['Uric_PH'] = np.random.uniform(4.5, 8.0, n_samples).round(1)
    data['Uric_specific_gravity'] = np.random.uniform(1.005, 1.035, n_samples).round(3)
    data['Uric_conductivity'] = np.random.uniform(8, 25, n_samples).round(0)
    data['Uric_epithelium'] = np.random.choice([0, 1, 2], n_samples, p=[0.7, 0.25, 0.05])
    data['Uric_bacteria'] = np.random.choice([0, 1, 2], n_samples, p=[0.8, 0.15, 0.05])
    
    # 血常规指标
    data['RBC'] = np.random.uniform(3.5, 5.5, n_samples).round(2)
    data['RDW.SD'] = np.random.uniform(35, 55, n_samples).round(0)
    data['RDW.CV'] = np.random.uniform(11.0, 16.0, n_samples).round(2)
    data['HCT'] = np.random.uniform(0.35, 0.50, n_samples).round(2)
    data['MCHC'] = np.random.uniform(320, 360, n_samples).round(0)
    data['MCH'] = np.random.uniform(26, 34, n_samples).round(0)
    data['MCV'] = np.random.uniform(80, 100, n_samples).round(0)
    data['HB'] = np.random.uniform(110, 170, n_samples).round(0)
    data['WBC'] = np.random.uniform(4.0, 10.0, n_samples).round(2)
    
    # 白细胞分类
    data['Neutrophil_count'] = np.random.uniform(2.0, 7.0, n_samples).round(2)
    data['Neutrophil_percentage'] = np.random.uniform(40, 70, n_samples).round(0)
    data['Lymphocytes_count'] = np.random.uniform(1.0, 4.0, n_samples).round(2)
    data['Lymphocytes_percentage'] = np.random.uniform(20, 45, n_samples).round(0)
    data['Basophil_count'] = np.random.uniform(0.01, 0.10, n_samples).round(2)
    data['Basophil_percentage'] = np.random.uniform(0.0, 1.5, n_samples).round(2)
    data['Eosinophil_count'] = np.random.uniform(0.05, 0.50, n_samples).round(2)
    data['Eosinophil_percentage'] = np.random.uniform(0.5, 5.0, n_samples).round(2)
    data['Monocytes_count'] = np.random.uniform(0.20, 0.80, n_samples).round(2)
    data['Monocytes_percentage'] = np.random.uniform(3, 10, n_samples).round(2)
    data['platelets_count'] = np.random.uniform(150, 400, n_samples).round(0)
    
    # 肝功能指标
    data['AST'] = np.random.uniform(10, 40, n_samples).round(0)
    data['ALT'] = np.random.uniform(10, 40, n_samples).round(0)
    data['AST_to_ALT_ratio'] = (data['AST'] / data['ALT']).round(2)
    data['DBIL'] = np.random.uniform(1, 8, n_samples).round(0)
    data['IBIL'] = np.random.uniform(5, 20, n_samples).round(0)
    data['TBIL'] = (data['DBIL'] + data['IBIL']).round(0)
    data['Globulin'] = np.random.uniform(20, 35, n_samples).round(0)
    data['Total_protein'] = np.random.uniform(65, 85, n_samples).round(0)
    data['Albumin'] = (data['Total_protein'] - data['Globulin']).round(0)
    data['ALB_to_GLO_ratio'] = (data['Albumin'] / data['Globulin']).round(2)
    
    # 其他生化指标
    data['CK'] = np.random.uniform(50, 200, n_samples).round(0)
    data['AKP'] = np.random.uniform(40, 150, n_samples).round(0)
    data['GGT'] = np.random.uniform(10, 60, n_samples).round(0)
    data['a_HBDH'] = np.random.uniform(80, 200, n_samples).round(0)
    data['LDH'] = np.random.uniform(120, 250, n_samples).round(0)
    data['Fasting_blood_glucose'] = np.random.uniform(3.9, 6.1, n_samples).round(2)
    
    # 血脂指标
    data['Cholesterol'] = np.random.uniform(3.0, 6.5, n_samples).round(2)
    data['Triglycerides'] = np.random.uniform(0.5, 2.5, n_samples).round(2)
    data['HDL'] = np.random.uniform(0.8, 2.0, n_samples).round(2)
    data['LDL'] = np.random.uniform(1.5, 4.5, n_samples).round(2)
    
    # 体征指标
    data['Systolic_BP'] = np.random.uniform(90, 160, n_samples).round(0)
    data['Diastolic_BP'] = np.random.uniform(60, 100, n_samples).round(0)
    data['Height'] = np.random.uniform(150, 185, n_samples).round(0)
    data['Weight'] = np.random.uniform(45, 95, n_samples).round(0)
    data['BMI'] = (data['Weight'] / (data['Height'] / 100) ** 2).round(2)
    data['Waistline'] = np.random.uniform(60, 110, n_samples).round(0)
    data['Hips'] = np.random.uniform(80, 120, n_samples).round(0)
    data['Waist_to_hip_ratio'] = (data['Waistline'] / data['Hips']).round(2)
    
    # 年龄
    data['Age'] = np.random.randint(18, 80, n_samples)
    
    # 计算衍生指标
    # eGFR计算 (简化的CKD-EPI公式)
    data['eGFR'] = np.clip(
        175 * (data['Creatinine'] / 88.4) ** -1.154 * data['Age'] ** -0.203,
        30, 120
    ).round(0)
    
    # 炎症比率指标
    data['NLR'] = (data['Neutrophil_count'] / data['Lymphocytes_count']).round(2)
    data['BLR'] = (data['Basophil_count'] / data['Lymphocytes_count']).round(2)
    data['ELR'] = (data['Eosinophil_count'] / data['Lymphocytes_count']).round(2)
    data['MLR'] = (data['Monocytes_count'] / data['Lymphocytes_count']).round(2)
    data['PLR'] = (data['platelets_count'] / data['Lymphocytes_count']).round(0)
    
    # 体型指标
    data['BRI'] = (data['Waistline'] / (data['Height'] * 0.01)).round(2)
    data['ABSI'] = (data['Waistline'] / (data['BMI'] ** (2/3) * data['Height'] ** 0.5)).round(2)
    
    # 分类变量
    for col, values in categorical_mappings.items():
        if col == 'stone':
            # 结石发生率约30%
            data[col] = np.random.choice(values, n_samples, p=[0.3, 0.7])
        elif col == 'Gender':
            # 性别均衡
            data[col] = np.random.choice(values, n_samples, p=[0.5, 0.5])
        else:
            # 其他分类变量
            weights = [0.4] + [0.6 / (len(values) - 1)] * (len(values) - 1)
            data[col] = np.random.choice(values, n_samples, p=weights)
    
    # 创建DataFrame
    df = pd.DataFrame(data)
    
    return df


# 生成数据
df = generate_medical_samples(n_samples=10000, random_state=42)

# 保存为Excel
df.to_excel('data_original/tianfu_baseline.xlsx', index=False)

print(f"成功生成 {len(df)} 条样本数据")
print(f"\n数据预览:")
print(df.head())
print(f"\n数据形状: {df.shape}")
print(f"\n结石分布:")
print(df['stone'].value_counts())

成功生成 10000 条样本数据

数据预览:
  id_patientarchive id_patient  Cystatin.C  Creatinine  Serum_Uric_acid  \
0        ARCH000001      P1001        0.94        73.0            405.0   
1        ARCH000002      P1002        1.46        70.0            215.0   
2        ARCH000003      P1003        1.26        58.0            271.0   
3        ARCH000004      P1004        1.14        91.0            382.0   
4        ARCH000005      P1005        0.74        81.0            319.0   

   Serum_Urea  Uric_RBC  Uric_WBC  Uric_PH  Uric_specific_gravity  ...  \
0        6.33         0         1      7.1                  1.006  ...   
1        5.26         0         0      7.6                  1.030  ...   
2        8.29         0         0      6.1                  1.012  ...   
3        3.81         0         1      5.5                  1.014  ...   
4        6.03         0         0      5.6                  1.012  ...   

   Alcohol  Gender  stone       Hypertension  Diabete  Stone_history      BPH  \